In [50]:
import os
import numpy as np
import pandas as pd
import cv2
import pickle

import csv
from sklearn.cluster import KMeans
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.utils import img_to_array  # Updated import
from keras.models import Model

In [8]:
# Path to your image dataset (200 folders)
IMAGE_DIR = os.path.join('..', 'Dataset', 'CUB_Dataset', 'CUB_Dataset', 'CUB_200_2011', 'images')
image_folder_path = IMAGE_DIR

# Parameters
IMG_SIZE = (224, 224)  # Image size for the CNN
N_CLUSTERS = 2 # Number of clusters

# Load pre-trained VGG16 model and remove the top layers
base_model = VGG16(weights='imagenet')
vgg_model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)


In [9]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMG_SIZE)  # Resize image
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)  # Preprocess for VGG16
    return image

In [30]:
%%time
def extract_features_from_image():
    
    # Extract features from images
    image_paths, features, class_labels, class_number = [], [], [], []

    for class_folder in os.listdir(image_folder_path):
        class_folder_path = os.path.join(image_folder_path, class_folder)
        # print(class_folder_path, class_folder_path.split('\\')[-1].split('.')[-1].replace('_', ' '))
        print(class_folder_path.split('\\')[-1])
        bird_lable = class_folder_path.split('\\')[-1].split('.')[-1].replace('_', ' ')
        bird_number = class_folder_path.split('\\')[-1].split('.')[0]

        # Ensure it's a directory
        if os.path.isdir(class_folder_path):
            for image_name in os.listdir(class_folder_path):
                image_path = os.path.join(class_folder_path, image_name)
                image_paths.append(image_path)  # Save the image path

                # Load and preprocess image
                image = load_and_preprocess_image(image_path)

                # Extract features
                feature = vgg_model.predict(image)
                features.append(feature.flatten())  # Flatten the feature array
                class_labels.append(bird_lable)
                class_number.append(bird_number)
                
    return (image_paths, features, class_labels, class_number)
    

CPU times: total: 0 ns
Wall time: 1.95 ms


In [47]:
def save_df(extract_features):
    features = np.array(extract_features[1])
    image_paths = np.array(extract_features[0])
    class_number = np.array(extract_features[3])
    class_labels = np.array(extract_features[2])

    df = pd.DataFrame(features)
    df['class_number'] = class_number
    df['class_labels'] = class_labels
    df['image_paths'] = image_paths
    display(df.head())
    
    CSV_PATH = os.path.join('..', 'Dataset', 'images_extract_features.csv')
    df.to_csv(CSV_PATH, index=False)
    
    print(f"File saved to : {CSV_PATH}")
    
def train_kmean_model(N_CLUSTERS, features):
    
    # Apply K-Means clustering
    kmeans = KMeans(n_clusters = N_CLUSTERS, random_state = 42)
    # clusters = kmeans.fit_predict(features) 
    # note : she using fit_predict which is wrong in this case need to use fit to train model
    kmeans.fit(features)
    
    return kmeans

# Function to save the model
def save_model(model, file_path):
    
    with open(file_path, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {file_path}")

# Function to load the model
def load_model(file_path):
    """
    Loads a model from a file.
    :param file_path: str, path of the saved model
    :return: loaded model
    """
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
    print(f"Model loaded from {file_path}")
    return model

# Function to predict with the loaded model
def predict(model, data):
    """
    Predicts the cluster labels for the given data using the model.
    :param model: trained KMeans model
    :param data: ndarray, data to predict
    :return: ndarray, cluster labels
    """
    return model.predict(data)

## I have few querys ? 

1. How can we that much sure about the split ?
2. What if : it not split / cluster properly ? (Need to find out)

## Lest find 

In [32]:
%%time
extract_features = extract_features_from_image()
print(len(extract_features))

001.Black_footed_Albatross
1/1 [==============================] - 0s 274ms/step
002.Laysan_Albatross
1/1 [==============================] - 0s 303ms/step
003.Sooty_Albatross
1/1 [==============================] - 0s 312ms/step


1/1 [==============================] - 0s 279ms/step
004.Groove_billed_Ani
1/1 [==============================] - 0s 288ms/step
005.Crested_Auklet
1/1 [==============================] - 0s 281ms/step
006.Least_Auklet
1/1 [==============================] - 0s 311ms/step


1/1 [==============================] - 1s 504ms/step
007.Parakeet_Auklet
1/1 [==============================] - 0s 360ms/step
008.Rhinoceros_Auklet
1/1 [==============================] - 0s 290ms/step
009.Brewer_Blackbird
1/1 [==============================] - 0s 280ms/step


1/1 [==============================] - 0s 275ms/step
010.Red_winged_Blackbird
1/1 [==============================] - 0s 300ms/step
011.Rusty_Blackbird
1/1 [==============================] - 0s 320ms/step
012.Yellow_headed_Blackbird
1/1 [==============================] - 0s 319ms/step


1/1 [==============================] - 0s 269ms/step
013.Bobolink
1/1 [==============================] - 0s 276ms/step
014.Indigo_Bunting
1/1 [==============================] - 0s 298ms/step


...

1/1 [==============================] - 0s 277ms/step
191.Red_headed_Woodpecker
1/1 [==============================] - 0s 281ms/step
192.Downy_Woodpecker
1/1 [==============================] - 0s 286ms/step
193.Bewick_Wren
1/1 [==============================] - 0s 276ms/step


1/1 [==============================] - 0s 346ms/step
194.Cactus_Wren
1/1 [==============================] - 0s 348ms/step
195.Carolina_Wren
1/1 [==============================] - 0s 284ms/step


1/1 [==============================] - 0s 300ms/step
196.House_Wren
1/1 [==============================] - 0s 302ms/step
197.Marsh_Wren
1/1 [==============================] - 0s 303ms/step
198.Rock_Wren
1/1 [==============================] - 0s 284ms/step


1/1 [==============================] - 0s 286ms/step
199.Winter_Wren
1/1 [==============================] - 0s 296ms/step
200.Common_Yellowthroat
1/1 [==============================] - 0s 283ms/step


1/1 [==============================] - 0s 302ms/step
4
CPU times: total: 3h 24min 52s
Wall time: 1h 10min 31s


In [48]:
%%time
# Save dataframe
save_df(extract_features)

,0,1,2,3,4,5,6,7,8,9,...,4089,4090,4091,4092,4093,4094,4095,class_number,class_labels,image_paths
0,3.042403,2.884922,0.000000,7.585155,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,001,Black footed Albatross,..\Dataset\CUB_Dataset\CUB_Dataset\CUB_200_201...
1,0.000000,0.000000,9.547824,5.867018,0.0,0.0,0.0,0.0,0.547732,0.614757,...,0.000000,0.0,0.0,13.213732,0.000000,0.0,0.0,001,Black footed Albatross,..\Dataset\CUB_Dataset\CUB_Dataset\CUB_200_201...
2,4.043432,0.000000,0.000000,1.765985,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,001,Black footed Albatross,..\Dataset\CUB_Dataset\CUB_Dataset\CUB_200_201...
3,0.000000,0.643451,0.000000,11.106439,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,3.763441,2.361611,0.0,0.0,001,Black footed Albatross,..\Dataset\CUB_Dataset\CUB_Dataset\CUB_200_201...
4,4.628634,3.541798,0.000000,2.993902,0.0,0.0,0.0,0.0,0.000000,0.000000,...,6.767929,0.0,0.0,1.820253,0.000000,0.0,0.0,001,Black footed Albatross,..\Dataset\CUB_Dataset\CUB_Dataset\CUB_200_201...


File saved to : ..\Dataset\images_extract_features.csv


C:\ProjectWork\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=42)


In [49]:
# Convert features to numpy array
features = np.array(extract_features[1])

# Train Kmean model
clusters = train_kmean_model(N_CLUSTERS, features)
print(clusters)

C:\ProjectWork\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2, random_state=42)
